In [1]:
import sys
sys.path.append('I:/YaoTony/code/x8313')
sys.path.append('I:/YaoTony/saleem tasks')
import Harville
#ys.path.append('/Users/saleemhuda/Dropbox/X8313_Live')
from datetime import date
import pytest
from pandas import date_range
import pandas as pd
import os.path

In [2]:
# dynamic strategy combination + scoring

import os
from pandas import DataFrame, concat, MultiIndex, pivot_table
from horse.betsim.wrap.daily import DailyRaces
from horse.betsim.wrap.jcapper import JCapper
from horse.betsim.wrap.bets import Bets
from datetime import datetime, timedelta
from experiments.fairvalue.models.history import map_jcp_itsp
from experiments.fairvalue.models.strategy import get_combinations_by_bet_type, generate_wagers, ProbModel
from experiments.fairvalue.models.probability import compute_probs_races,  add_win_probs_from_score
from experiments.fairvalue.models.selector import SelectMinimumRunners
from experiments.fairvalue.models.strategy import Strategy, ProbModel
from horse.betsim.data.db.engines import engine_ac
from experiments.fairvalue.models.history import History, LabelHistory
from experiments.fairvalue.models.probability import generate_permutation_matrix, HarvilleTri , MultiProbability, multiprob_base, multiprob_harville
#from experiments.fairvalue.models.weight import
import scipy.stats as ss
import horse

import itertools
import pandas as pd
import os

# SCRIPT GLOBALS

# output path
horse_path = os.path.dirname(horse.__file__)
path_project = os.path.dirname(horse_path)
path_scratch = os.path.join(path_project, 'scratch')
path_live = os.path.join('/Users/saleemhuda/Dropbox/X8313_LIVE/')
# target date
today = datetime.today()
yesterday = today - timedelta(days=1)

history = History(engine_ac, verbose=True)
# get sim dates from history.match_wom_dates(today)
#dates_history = history.match_wom_dates(today, start_year=2015, end_year=2016, filter_month=True)
dates_history = [date(2016,7,3)]
history.load(dates_history)
# add computed columns and factors
target_date_format = today.strftime("%Y%m%d")
hist_date_format= dates_history[0].strftime("%Y%m%d")
from horse.betsim import data
import os.path
track_detail = os.path.join(data.__path__._path[0], 'track_detail.csv')
df_track_detail = pd.read_csv(track_detail, index_col=['x8_track_sym'])
map_jcp_itsp = df_track_detail.set_index('jcp_track_sym')['itsp_track_sym'].to_dict()
map_rebates_win = df_track_detail.WN.to_dict()

lbh = LabelHistory()
lbh.load(history)

dfX_hist = lbh.dfX.copy()

 0. a1-a1-win-1
 1. a1-a1-win-2
 2. a1-a1-win-3
 3. a1-a1-win-4
 4. a1-a1-wildcard-1
 5. a1-a1-wildcard-2
 6. a1-a1-wildcard-3
 7. a1-a1-wildcard-4
 8. a1-a2-win-1
 9. a1-a2-win-2
10. a1-a2-win-3
11. a1-a2-win-4
12. a1-a2-wildcard-1
13. a1-a2-wildcard-2
14. a1-a2-wildcard-3
15. a1-a2-wildcard-4
16. a1-wildcard-win-1
17. a1-wildcard-win-2
18. a1-wildcard-win-3
19. a1-wildcard-win-4
20. a1-wildcard-wildcard-1
21. a1-wildcard-wildcard-2
22. a1-wildcard-wildcard-3
23. a1-wildcard-wildcard-4
24. a1-win-win-1
25. a1-win-win-2
26. a1-win-win-3
27. a1-win-win-4
28. a1-win-wildcard-1
29. a1-win-wildcard-2
30. a1-win-wildcard-3
31. a1-win-wildcard-4
32. place-a1-win-1
33. place-a1-win-2
34. place-a1-win-3
35. place-a1-win-4
36. place-a1-wildcard-1
37. place-a1-wildcard-2
38. place-a1-wildcard-3
39. place-a1-wildcard-4
40. place-a2-win-1
41. place-a2-win-2
42. place-a2-win-3
43. place-a2-win-4
44. place-a2-wildcard-1
45. place-a2-wildcard-2
46. place-a2-wildcard-3
47. place-a2-wildcard-4
48. plac

added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_norm_par
added prob_x8runner_HDWPSRRating_n

In [3]:
dfX_hist = dfX_hist[dfX_hist.x8_num_starters_pp>5].copy()
dfX_hist['payout_win'] = dfX_hist['final_tote_odds']+1
dfX_hist['rebate_win'] = dfX_hist['track_id'].map(lambda x:map_rebates_win.get(x,0.0))
dfX_hist.set_index(['race_id','runner_id','x8name_runner'], inplace=True)

In [4]:
def _get_calibration_data(_lbh, tracks=None):
    """
    Puts payout , pool data together with runner probabiility data
    Needs probabilities, final_tote_odds, poolsize, payout, runners
    :return:
    :rtype:
    """
    # merge runner factor data in dfX with payout data
    # then merge pool data
    dfmergepaydfx = pd.merge(_lbh.df_pivot_payout.reset_index(), _lbh.dfX, left_on=['race_id'], right_on=['race_id'],
                             how='right')
    df_calibrate = pd.merge(dfmergepaydfx, _lbh.df_pivot_pool.reset_index(), left_on=['race_id'],
                                 right_on=['race_id'], how='right', suffixes=('_payout','_pool'))
    return df_calibrate


In [5]:
df_CALIBRATE = _get_calibration_data(lbh)

In [ ]:
df_CALIBRATE.official_finish_position

In [14]:
df_CALIBRATE.head()

,race_id,EX_payout,PL_payout,SH_payout,SU_payout,TR_payout,WN_payout,index,track_id,date,...,year,month,weekday,prob_x8runner_HDWPSRRating_norm_par,EX_pool,PL_pool,SH_pool,SU_pool,TR_pool,WN_pool
0,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,98.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.137261,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
1,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,99.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.083753,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
2,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,100.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.471878,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
3,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,101.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.137261,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0
4,ALB_20160703_10,13.3,2.35,1.533333,259.0,56.9,2.4,102.0,ALB,2016-07-03,...,2016.0,7.0,6.0,0.000223,10188.0,12167.0,12167.0,6432.0,8624.0,12167.0


In [ ]:
#df_CALIBRATE = pd.read_csv('./DF_CALIBRATE_20180629.csv')
#df_CALIBRATE.head()

In [17]:
df = df_CALIBRATE[['race_id','EX_payout','runner_program_number_pp','x8is_exacta','official_finish_position','prob_x8runner_HDWPSRRating_norm_par']]
df = df.sort_values(by=['race_id','official_finish_position']).dropna(subset=['EX_payout'])
df.head()

,race_id,EX_payout,runner_program_number_pp,x8is_exacta,official_finish_position,prob_x8runner_HDWPSRRating_norm_par
0,ALB_20160703_10,13.3,2,1.0,1.0,0.137261
1,ALB_20160703_10,13.3,1,1.0,2.0,0.083753
2,ALB_20160703_10,13.3,8,0.0,3.0,0.471878
3,ALB_20160703_10,13.3,7,0.0,4.0,0.137261
4,ALB_20160703_10,13.3,3,0.0,5.0,0.000223


In [22]:
df.to_csv('DF_CALIBRATE_20180630.csv', index=False)

In [18]:
#exacta bet type
import numpy as np
df_exacta = df[np.isfinite(df['prob_x8runner_HDWPSRRating_norm_par'])]#delete rows when race only has one runner
df_exacta['is_paid']=0
race_set = set(df_exacta['race_id'])
for race in race_set:
    df_exacta_sub = df_exacta[df_exacta['race_id']==race]
    #fav is prob ranking 1 of exacta combination and its prob
    fav = Harville.harville_multiprob(list(df_exacta_sub['runner_program_number_pp']),list(df_exacta_sub['prob_x8runner_HDWPSRRating_norm_par']),'Exacta')[0]
    df_exacta_sub_ex = df_exacta_sub[df_exacta_sub['x8is_exacta']>0]#extract extacta runners
    df_exacta_sub_ex=df_exacta_sub_ex.reset_index(drop=True)
    #suppose is_paid is 0 if only one runner participate the race
    if len(df_exacta_sub_ex)>1:
        if df_exacta_sub_ex.loc[1,'official_finish_position']==1:
            if set(fav[0:2]).issubset(set(df_exacta_sub_ex['runner_program_number_pp'])):
                df_exacta.loc[df_exacta['race_id']==race,'is_paid']=1
        elif list(df_exacta_sub_ex['runner_program_number_pp'])==fav[0:2]:
            df_exacta.loc[df_exacta['race_id']==race,'is_paid']=1


/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [19]:
df_exacta[df_exacta['is_paid']==1]

,race_id,EX_payout,runner_program_number_pp,x8is_exacta,official_finish_position,prob_x8runner_HDWPSRRating_norm_par,is_paid
85,APX_20160703_7,11.2,1,1.0,1.0,0.191468,1
86,APX_20160703_7,11.2,8,1.0,2.0,0.172299,1
87,APX_20160703_7,11.2,4,0.0,3.0,0.123758,1
88,APX_20160703_7,11.2,5,0.0,4.0,0.098022,1
89,APX_20160703_7,11.2,10,0.0,5.0,0.000030,1
90,APX_20160703_7,11.2,2,NaN,NaN,0.086895,1
91,APX_20160703_7,11.2,3,NaN,NaN,0.123758,1
92,APX_20160703_7,11.2,6,NaN,NaN,0.029978,1
93,APX_20160703_7,11.2,7,NaN,NaN,0.086895,1
94,APX_20160703_7,11.2,9,NaN,NaN,0.086895,1


In [20]:
#exacta box bet type, Quinella
import numpy as np

df_quinella = df[np.isfinite(df['prob_x8runner_HDWPSRRating_norm_par'])]#delete rows when race only has one runner
df_quinella['is_paid']=0
race_set = set(df_quinella['race_id'])
for race in race_set:
    df_quinella_sub = df_quinella[df_quinella['race_id']==race]
    #fav is prob ranking 1 of box combination and its prob
    fav = Harville.harville_box(list(df_quinella_sub['runner_program_number_pp']),list(df_quinella_sub['prob_x8runner_HDWPSRRating_norm_par']),'Exacta')[0]
    # we can also use fav = Harville.harville_multiprob(list(df_quinella_sub['runner_program_number_pp']),list(df_quinella_sub['prob_x8runner_HDWPSRRating_norm_par']),'Quinella')[0]
    
    df_quinella_sub_ex = df_quinella_sub[df_quinella_sub['x8is_exacta']>0]#extract extacta runners
    if set(fav[0:2]).issubset(set(df_quinella_sub_ex['runner_program_number_pp'])):
        df_quinella.loc[df_quinella['race_id']==race,'is_paid']=1

/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/Users/saleemhuda/miniconda3/envs/env_x8/lib/python3.5/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [21]:
df_quinella[df_quinella['is_paid']==1]

,race_id,EX_payout,runner_program_number_pp,x8is_exacta,official_finish_position,prob_x8runner_HDWPSRRating_norm_par,is_paid
85,APX_20160703_7,11.20,1,1.0,1.0,0.191468,1
86,APX_20160703_7,11.20,8,1.0,2.0,0.172299,1
87,APX_20160703_7,11.20,4,0.0,3.0,0.123758,1
88,APX_20160703_7,11.20,5,0.0,4.0,0.098022,1
89,APX_20160703_7,11.20,10,0.0,5.0,0.000030,1
90,APX_20160703_7,11.20,2,NaN,NaN,0.086895,1
91,APX_20160703_7,11.20,3,NaN,NaN,0.123758,1
92,APX_20160703_7,11.20,6,NaN,NaN,0.029978,1
93,APX_20160703_7,11.20,7,NaN,NaN,0.086895,1
94,APX_20160703_7,11.20,9,NaN,NaN,0.086895,1
